In [2]:
"""
The following multi-layer perceptron builds heavily on two sources:
Blog: A Step by Step Backpropagation Example - https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
Book: Machine learning, An algorithmic perspective 2nd Edition by Stephen Marsland

At the end is the run section, run it with the default values from the blog. No external files needed to run.
"""
from __future__ import division
import numpy as np
import math

class neural_network:
    def __init__(self, weights_hidden, weights_output, bias_hidden_weight, bias_output_weight, n_hidden_layers=2, learning_rate=0.5):
        self.weights_hidden = weights_hidden
        self.weights_output = weights_output
        self.bias_hidden_weight = np.array([bias_hidden_weight] * 2)
        self.bias_output_weight = np.array([bias_output_weight] * 2)
        self.n_hidden_layers = n_hidden_layers
        self.learning_rate = learning_rate
        self.beta = 1
        
        self.hidden_layer_activation = np.array([0.,0.])
        self.hidden_layer_activation_f_output = np.array([0.,0.])
        self.hidden_layer_error = np.array([0.,0.])
        self.output_layer_output = np.array([0.,0.])
        self.output_layer_activation = np.array([0.,0.])
        self.output_layer_error = np.array([0., 0.])
        
        self.all_total_errors = np.array([])
    
    #activation function - sigmoid is acceptable when having a classification problem
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x * self.beta))

    ##calculate hidden layer activations and outputs from activation function
    def forward_hidden_layer(self): 
        for i in range(0, self.n_hidden_layers):
            curr_weights = self.weights_hidden[i,:]
            
            #calculate hidden neuron value
            current_hidden_neutron = np.dot(self.inputs, curr_weights) + self.bias_hidden_weight[i]
            np.put(self.hidden_layer_activation, i, current_hidden_neutron)
            np.put(self.hidden_layer_activation_f_output, i, self.sigmoid(current_hidden_neutron))

    ##calculate output layer
    def forward_output_layer(self):
        n_target_neurons = self.targets.shape[0] #get number og output neurons (elements in target)
        for i in range(0, n_target_neurons):
            #print("--- Calculate output neuron number:", i)
            curr_weights = self.weights_output[i,:]
            #calculate output neuron value
            current_output_neutron = np.dot(self.hidden_layer_activation_f_output, curr_weights) + self.bias_output_weight[i]
            np.put(self.output_layer_output, i, current_output_neutron)
            np.put(self.output_layer_activation, i, self.sigmoid(current_output_neutron))
    
    def calculate_total_error(self, counter):
        total_error = np.sum(1/2 * (self.targets - self.output_layer_activation) ** 2)
        if (counter % 100 == 0 or counter == 1): #add only every 100th neural network error to the array
            self.all_total_errors = np.append(self.all_total_errors, total_error)
    
    def calculate_output_error(self):
        ##compute the error at the output
        self.output_layer_error = (self.output_layer_activation - self.targets) * self.output_layer_activation * (1 - self.output_layer_activation) 
         
    ##calculate the error in the hidden layer
    def backward_hidden_error(self):
        #first part of the equation - the derivative of the activation function - (x*(1-x))
        derivative_part = self.hidden_layer_activation_f_output * (1 - self.hidden_layer_activation_f_output)
        #second part of the equation - sum of product of weights in hidden layer and the error of the output
        sum_part = self.weights_hidden * self.output_layer_error        
        self.hidden_layer_error = derivative_part * sum_part
        
    def update_output_layer_weights(self):
        self.weights_output = self.weights_output - (self.learning_rate * self.output_layer_error * self.hidden_layer_activation_f_output)
        self.bias_output_weight = self.bias_output_weight - (self.learning_rate * self.output_layer_error * 1)
        
    def update_hidden_layer_weights(self):
        self.weights_hidden =  self.weights_hidden - (self.learning_rate * self.hidden_layer_error * self.inputs)
        self.bias_hidden_weight = self.bias_hidden_weight - (self.learning_rate * self.hidden_layer_activation * 1)
    
    ##train method
    def train(self, inputs, targets, n_runs):
        self.inputs = inputs
        self.targets = targets
        self.n_runs = n_runs
        
        print("input values:\n", self.inputs)
        print("initial HIDDEN layer weights:\n", self.weights_hidden)
        print("initial HIDDEN layer bias weights:\n", self.bias_hidden_weight)
        print("initial OUTPUT layer weights:\n", self.weights_output)
        print("initial OUTPUT layer bias weights:\n", self.bias_output_weight)
        print("========") 
        
        for i in range(1, n_runs + 1):
            ##################
            ## Forward pass #
            ################
            self.forward_hidden_layer()
            self.forward_output_layer()      
            self.calculate_total_error(i)
            
            ###################
            ## Backward pass #
            #################
            self.calculate_output_error()
            self.backward_hidden_error()
            
            self.update_output_layer_weights()
            self.update_hidden_layer_weights() 
        
    #end train    
    
    ##print an output with final information about the learning of neural network
    def get_final_output(self):
        print("==============\n==============")
        print("\nOutput after training with {} runs:".format(self.n_runs))
        print("Total error of the neural network after each run:\n", self.all_total_errors)
        print("Targets:\n", self.targets)
        print("Final neural network outputs:\n", self.output_layer_activation)
    
#end class neural_network        


#########
## Run #
#######

#original from the blog
X = np.array([0.05, 0.1])
target= np.array([0.01, 0.99])

weights_hidden_layer = np.array([[0.15, 0.2], [0.25, 0.3]])
weights_output_layer = np.array([[0.4, 0.45], [0.5, 0.55]])
bias_hidden_weight = 0.35
bias_output_weight = 0.6
number_of_training_runs = 10000

nn = neural_network(weights_hidden_layer, weights_output_layer, bias_hidden_weight, bias_output_weight, learning_rate=0.5)
nn.train(X, target, number_of_training_runs)
nn.get_final_output()

input values:
 [ 0.05  0.1 ]
initial HIDDEN layer weights:
 [[ 0.15  0.2 ]
 [ 0.25  0.3 ]]
initial HIDDEN layer bias weights:
 [ 0.35  0.35]
initial OUTPUT layer weights:
 [[ 0.4   0.45]
 [ 0.5   0.55]]
initial OUTPUT layer bias weights:
 [ 0.6  0.6]

Output after training with 10000 runs:
Total error of the neural network after each run:
 [  2.98371109e-01   1.34911767e-02   5.74197301e-03   3.47777194e-03
   2.43212527e-03   1.83954926e-03   1.46180009e-03   1.20179705e-03
   1.01289869e-03   8.70034480e-04   7.58579749e-04   6.69453801e-04
   5.96733501e-04   5.36397991e-04   4.85626153e-04   4.42382898e-04
   4.05164981e-04   3.72839070e-04   3.44535347e-04   3.19575681e-04
   2.97423979e-04   2.77651134e-04   2.59909821e-04   2.43916062e-04
   2.29435536e-04   2.16273278e-04   2.04265818e-04   1.93275137e-04
   1.83183936e-04   1.73891936e-04   1.65312923e-04   1.57372395e-04
   1.50005658e-04   1.43156280e-04   1.36774831e-04   1.30817836e-04
   1.25246920e-04   1.20028086e-04   